In [ ]:
#This program was used to filter tweets on basis of location and tags and then cleaning them

In [1]:
import csv
import json
import pandas as pd
import numpy as np
import re
import nltk
import time
import matplotlib.pyplot as plt
from pprint import pprint
from ast import literal_eval
from langdetect import detect_langs
from langdetect import DetectorFactory
DetectorFactory.seed=0
import fun_

KeyboardInterrupt: 

In [ ]:
#Obtaining dictionary list from json file
dict_list=[]
start_t=time.time()

count=0
with open('file.json','r') as f:
    for el in f:
        count=count+1
        try:
            dict_= json.loads(el)
            dict_list.append(dict_)
        except:
            #dict_list.append(dict_)
            print(dict_['id'], count)
            #raise
            continue

print('No of Tweets at input:',len (dict_list))        
print('time till forming list of dictionaries from json:',time.time()-start_t)

In [ ]:
#forming dataframe from list of dict
df=pd.DataFrame()
for fld in dict_list[0].keys():
    if fld!="extended_tweet" and fld!="display_text_range": #because these 2 fields aren't necessarly present in a tweet
                                                            #thus they are handled seperately after this loop
        df[fld]=list(map(lambda arg:arg[fld],dict_list )) 
        
df['display_text_range']=list(map(lambda arg: literal_eval(arg['display_text_range']) if 'display_text_range' in arg.keys() else None , dict_list ))
p=list(map( lambda arg: literal_eval( arg['extended_tweet']) if 'extended_tweet' in arg.keys() else None , dict_list ))
df['full_text']= list(map(lambda arg: arg['full_text'] if arg!=None else None , p ))
df['display_full_text_range']=list(map(lambda arg: (arg['display_text_range']) if arg!=None else None , p))

#adding extra fields that will be used later 
df['translate']=False   #this field tells weather a tweet is in local language and thus needs translation
df['trans_text']='NONE' #contains translated text (In this project no translation was carried out)
df['pros_text']='NONE'  #processed text (text after cleaning)
df['trans_lang']='NONE' #detected language

print('time till forming dataframe from list of dict:',time.time()-start_t)

In [ ]:
#filtering by location (local matching)
x= df.apply(fun_.fun_loc, axis=1)
logic=list(map( lambda arg: arg[0], x ))
trn=list(map( lambda arg: arg[1], x ))
logic_=list(map( lambda arg: not arg, logic ))

fil_by_loc= df.loc[(df.index[logic])]
fil_by_loc['translate']=list(filter( None.__ne__,trn)) 
rem=df.loc[(df.index[logic_])] #rem means remaining tweets (which were not accepted by above location filter)
print('time till filtering by location:',time.time()-start_t)

In [ ]:
#filtering by tags (this is done on remaining tweets from above filter)
x=rem.apply(fun_.fun_text, axis=1)
logic=list(map( lambda arg: arg[0], x ))
trn=list(map( lambda arg: arg[1], x ))
logic_=list(map( lambda arg: not arg, logic ))

fil_by_tag=rem.loc[(rem.index[logic])]
fil_by_tag['translate']=list(filter( None.__ne__,trn))
rem2=rem.loc[(rem.index[logic_])]
print('time till filtering by tags:',time.time()-start_t)

In [ ]:
#Tweets filtered by tags are further filtered to remove tweets related to foreign tags (like Brexit, US congress etc)
logic=fil_by_tag.apply(fun_.fun_foreign , axis=1)
logic_=list(map( lambda arg: not arg, logic ))

mod_fil_by_tag=fil_by_tag.loc[(fil_by_tag.index[logic])]
rem3=fil_by_tag.loc[(fil_by_tag.index[logic_])]
print('time till removing foreign tags:',time.time()-start_t)

In [ ]:
#cleaning tweets filtered by location and tag
fil_by_loc['pros_text']=fil_by_loc.apply(fun_.f_clean1,axis=1)
mod_fil_by_tag['pros_text']=mod_fil_by_tag.apply(fun_.f_clean1,axis=1)
print('time till clean1:',time.time()-start_t)

In [ ]:
fil_by_loc.to_csv(r'tweetsByLoc.csv',index=None,header=True)
mod_fil_by_text.to_csv(r'tweetsByTag.csv',index=None,header=True)
rem2.to_csv(r'rem2.csv',index=None,header=True)
rem3.to_csv(r'rem3.csv',index=None,header=True)  